In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
import torch.nn as nn
from importlib import reload
import MobileNet_v3
import functions
MobileNet_v3 = reload(MobileNet_v3)

functions = reload(functions)
from functions import train, accuracy, MyDataLoader, train_test_split
from MobileNet_v3 import get_model as gm3


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()

In [ ]:
mobilenet = gm3().to(DEVICE)

In [ ]:
# dataset settings
batch_size = 32

train_transformer = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize([0.24, 0.25, 0.26], [.31, .32, .33])
])

cifar_train = datasets.CIFAR100('data/',
                             transform=train_transformer,
                             download=True)
cifar_test = datasets.CIFAR100('data/', train=False)

train_indices, val_indices = train_test_split(np.arange(len(cifar_train)), .75, cifar_train.targets)
train_loader = MyDataLoader(cifar_train, batch_size, train_indices, shuffle=True)
val_loader = MyDataLoader(cifar_train, batch_size, val_indices, shuffle=True)
test_loader = MyDataLoader(cifar_test, batch_size, shuffle=False)

In [ ]:
# optimizer settings
learning_rate = 1e-1

# train settings
epochs_count = 3


optimizer = torch.optim.RMSprop(mobilenet.parameters(),
                                lr=learning_rate,
                                momentum=0.9,
                                weight_decay = 1e-5
                            )
loss_func = nn.NLLLoss()


In [ ]:
train_history, best_parameters = \
    train(mobilenet, train_loader, loss_func, optimizer, epochs_count, accuracy, val_loader)